In [34]:
import csv
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [43]:
def scrape_review(file_path, num_pages, link):

# file_path - File path for review output
# num_pages - Number of pages to iterate and scrape
# link - Link to webpage to scrape

  # open the file to save the review
  file = open(file_path, 'a', encoding="utf-8", newline='')
  csv_writer = csv.writer(file)

  # Open up browser and navigate to page
  driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
  driver.get(link)

  for i in range(num_pages):
    
    # Let page load (change to until element exist later)
    time.sleep(10)
    
    # Extract elements               
    reviews = driver.find_elements(By.XPATH, "//div[@data-automation='reviewCard']/div[5]/div[1]")
    dates = driver.find_elements(By.XPATH, "//*[@class='eRduX']")
    
    # Open csv file and write to it
    for j in range(len(reviews)):
      review = reviews[j].text.replace("\n", "")
      date = dates[j].text[:9].strip()
      csv_writer.writerow([date, review]) 
      
    next = driver.find_element(By.XPATH, "//a[@aria-label='Next page']//*[name()='svg']")
    next.click()
    
  file.close()

  driver.close()

In [44]:
# Test run on 1st 3 pages
scrape_review('data/raw/ta_sea_aquarium_raw.csv', 3, "https://www.tripadvisor.co.id/Attraction_Review-g294264-d4009739-Reviews-S_E_A_Aquarium-Sentosa_Island.html")



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\Luqman\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


# Review Counter

In [ ]:
attractions = ['https://www.tripadvisor.co.id/Attraction_Review-g294264-d2439664-Reviews-Universal_Studios_Singapore-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d4009739-Reviews-S_E_A_Aquarium-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d3747640-Reviews-Adventure_Cove_Waterpark-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d3475270-Reviews-Sentosa_Boardwalk-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d7178019-Reviews-Madame_Tussauds_Singapore-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d1997266-Reviews-Singapore_Cable_Car_Sentosa-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d941828-Reviews-Palawan_Beach-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d1626904-Reviews-Skyline_Luge_Sentosa-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d6714698-Reviews-Trick_Eye_Museum_Singapore-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d1758910-Reviews-Mega_Adventure-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d2180413-Reviews-IFly_Singapore-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d1936430-Reviews-Sentosa_4D_Adventureland-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294265-d12795233-Reviews-Skypark_Sentosa_by_AJ_Hackett-Singapore.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d19944077-Reviews-HydroDash-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d1891492-Reviews-Siloso_Beach-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d324747-Reviews-Fort_Siloso-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d1891491-Reviews-Tanjong_Beach-Sentosa_Island.html',
              'https://www.tripadvisor.co.id/Attraction_Review-g294264-d1997267-Reviews-Sentosa_Nature_Discovery-Sentosa_Island.html']

In [52]:
# Version 2

def scrape_review_v2(num_pages, link):

  # Open up browser and navigate to page
  driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
  driver.get(link)
  
  review_count = 0
  page_num = 1
  
  for i in range(num_pages):
    
    # Let page load (change to until element exist later)
    time.sleep(5)
    
    # Extract elements
    container = driver.find_elements(By.XPATH, "//*[@data-automation='reviewCard']//span[contains(text(),'Google Terjemahan')]")           
    
    review_count += len(container)
          
    next = driver.find_element(By.XPATH, "//a[@aria-label='Next page']//*[name()='svg']")
    next.click()
    page_num += 1
        
    # Open csv file and write to it
    # for j in range(len(reviews)):
    #   review = container[j].find_element(By.XPATH, "")
    #   date = dates[j].text[:9].strip()
    #   csv_writer.writerow([date, review]) 
      
    print(f"Review Count: {review_count}")
    print(f"Current page: {page_num}")
    
  # file.close()
  print(review_count)
  driver.close()

In [53]:
# Version 2
# Test run
scrape_review_v2(15, "https://www.tripadvisor.co.id/Attraction_Review-g294264-d4009739-Reviews-S_E_A_Aquarium-Sentosa_Island.html")



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\Luqman\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


Review Count: 10
Current page: 2
Review Count: 20
Current page: 3
Review Count: 30
Current page: 4
Review Count: 40
Current page: 5
Review Count: 50
Current page: 6
Review Count: 60
Current page: 7
Review Count: 70
Current page: 8
Review Count: 80
Current page: 9
Review Count: 90
Current page: 10
Review Count: 100
Current page: 11
Review Count: 110
Current page: 12
Review Count: 120
Current page: 13
Review Count: 130
Current page: 14
Review Count: 140
Current page: 15
Review Count: 143
Current page: 16
143
